In [1]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model
from flows.models.utils import test_flow

# CIFAR10

In [4]:
import models
architecture = getattr(models, "LinearOneLayer60k") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    False)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/LinearOneLayer60k/curve1/checkpoint-300.pt')['model_state'])
model2.load_state_dict(torch.load('curves/LinearOneLayer60k/curve2/checkpoint-300.pt')['model_state'])

Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [9]:
te_list = []
tr_list = []
for i in range(1, 3):
    path = 'curves/LinearOneLayer60k/curve'+str(i)+'/checkpoint-300.pt'
    model1.load_state_dict(torch.load(path)['model_state'])
    model1.cuda()
    tr, te = test_model(model1, loaders, cuda=True)
    te_list.append(te['accuracy'])
    tr_list.append(tr['accuracy'])
    
te_list = np.array(te_list)
tr_list = np.array(tr_list)

train results {'nll': 0.7848174110794067, 'loss': 0.7848174110794067, 'accuracy': 73.118}
test results {'nll': 1.3959318704605101, 'loss': 1.3959318704605101, 'accuracy': 60.2}
train results {'nll': 0.7854795127296448, 'loss': 0.7854795127296448, 'accuracy': 73.164}
test results {'nll': 1.3850665266036988, 'loss': 1.3850665266036988, 'accuracy': 60.62}


In [10]:
te_list.mean(), te_list.std(), tr_list.mean(), tr_list.std()

(60.41, 0.2099999999999973, 73.14099999999999, 0.02300000000000324)

# Computing mean and std for different methods on CIFAR10

## basic 

In [3]:
from one_layer_utils import samples, get_b
test_dataset = [samples(model1), samples(model2)]
b = get_b(model1, model2)

ModuleNotFoundError: No module named 'one_layer_utils'

In [6]:
cntr = Connector(None, None)

In [1]:
funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func 
         and 'flow' not in func and 'third_cumulant' not in func and 'inverse_connect' not in func]

NameError: name 'cntr' is not defined

In [2]:
funcs

NameError: name 'funcs' is not defined

In [12]:
test_models(test_dataset, loaders, b, architecture, flow=None, K=300, t=0.5,
                show=False, verbose=True, func_name=funcs)

arc_connect
0.5
train results {'nll': 1.5711080474472046, 'loss': 1.5711080474472046, 'accuracy': 54.714}
test results {'nll': 2.7822178539276123, 'loss': 2.7822178539276123, 'accuracy': 44.61}
arc_connect_PCA
0.5
train results {'nll': 1.5660534892272948, 'loss': 1.5660534892272948, 'accuracy': 54.63}
test results {'nll': 2.7822229469299318, 'loss': 2.7822229469299318, 'accuracy': 44.61}
inverse_connect


Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/opt/all/anaconda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/all/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


KeyboardInterrupt: 

  File "/opt/all/anaconda/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/opt/all/anaconda/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/ivan/distribution_connector/connector.py", line 52, in f
    u.append(distr.F(x))
  File "/home/ivan/distribution_connector/connector.py", line 23, in F
    b = self.sorted_X >= x
KeyboardInterrupt
